In [1]:
import gym

env = gym.make('Pong-v4',render_mode='rgb_array')
observation = env.reset()


A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


In [2]:
# Render the game state
frame = env.render()
import cv2

# Convert to grayscale and resize to a lower resolution
gray_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
small_frame = cv2.resize(gray_frame, (84, 84))  # Resize to 84x84 for simplicity


/Users/kpradjinata/Documents/Research/OpenAIGym/venv/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:289: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


In [3]:
# Flatten image for the LLM input (example)
input_to_llm = small_frame.flatten()


In [4]:
description = "Ball is at (50, 30), player paddle is at y = 40, opponent paddle is at y = 60."
# action_text = predict_action_llm(description)
# print(f"LLM suggests action: {action_text}")


In [5]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
# Load the GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Check if MPS is available and use it
device = torch.device("mps" if torch.has_mps else "cpu")
model.to(device)

def predict_action_llm(description):
    input_ids = tokenizer.encode(description, return_tensors='pt', truncation=True).to(device)  # No max_length here
    output = model.generate(input_ids, max_new_tokens=30)  # Control the output length
    action_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return action_text

device = torch.device("mps" if torch.has_mps else "cpu")

print(f"MPS available: {torch.has_mps}")


/Users/kpradjinata/Documents/Research/OpenAIGym/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/kpradjinata/Documents/Research/OpenAIGym/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/var/folders/kh/kdb6f8j523s5fh4syx6bp5bh0000gn/T/ipykernel_5530/3070159875.py:8: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  device = torch.device("mps" if torch.has_mps else "cpu")


MPS available: True


/var/folders/kh/kdb6f8j523s5fh4syx6bp5bh0000gn/T/ipykernel_5530/3070159875.py:17: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  device = torch.device("mps" if torch.has_mps else "cpu")
/var/folders/kh/kdb6f8j523s5fh4syx6bp5bh0000gn/T/ipykernel_5530/3070159875.py:19: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  print(f"MPS available: {torch.has_mps}")


In [6]:
import cv2
import numpy as np

def describe_game_state(frame):
    # Convert the frame to grayscale (if needed) and threshold to identify objects
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    _, thresh_frame = cv2.threshold(gray_frame, 100, 255, cv2.THRESH_BINARY)
    
    # Find contours which could represent objects in the frame
    contours, _ = cv2.findContours(thresh_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Placeholder values for positions
    ball_x, ball_y = -1, -1
    player_paddle_y = -1
    opponent_paddle_y = -1
    
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        
        # You may need to fine-tune these conditions to match actual Pong objects
        if w > 10 and h > 10:  # Assuming the ball is smaller than the paddles
            ball_x, ball_y = x, y
        elif x < frame.shape[1] // 2:  # Left side: player paddle
            player_paddle_y = y
        else:  # Right side: opponent paddle
            opponent_paddle_y = y
    
    # Return a text description of the game state
    description = (f"Ball is at ({ball_x}, {ball_y}), "
                   f"player paddle is at y = {player_paddle_y}, "
                   f"opponent paddle is at y = {opponent_paddle_y}.")
    
    return description


In [7]:
def map_action(action_text):
    if "up" in action_text.lower():
        return 2  # Move paddle up
    elif "down" in action_text.lower():
        return 3  # Move paddle down
    else:
        return 0  # Stay still


In [8]:
# step = 0
# observation = env.reset()

# while True:
#     step += 1
    
#     # Step the environment with the current action and get the next frame
#     observation, reward, done, info, truncation = env.step(action)
    
#     if step % 10 == 0:  # Call LLM only every 10 frames
#         frame = env.render()
#         description = describe_game_state(frame)
#         print(description)
        
#         action_text = predict_action_llm(description)
#         action = map_action(action_text)
#     else:
#         action = 0  # Default action when not calling the LLM
    
#     # Check if the episode is done (ball out of bounds, etc.)
#     if done:
#         observation = env.reset()  # Reset the environment if the episode ends
#         step = 0


In [9]:
import gym
import cv2
import numpy as np

# Initialize the environment
env = gym.make('Pong-v4', render_mode='rgb_array')
observation = env.reset()

def detect_ball(frame):
    # Convert to grayscale for easier processing
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    
    # Apply threshold to identify bright objects (like the ball)
    _, thresh_frame = cv2.threshold(gray_frame, 200, 255, cv2.THRESH_BINARY)
    
    # Find contours, which could represent objects (like the ball)
    contours, _ = cv2.findContours(thresh_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Loop over the contours and find the ball based on size
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        # Assuming the ball is small compared to paddles, filter based on size
        if w < 10 and h < 10:  # Adjust these values based on the actual ball size
            return (x, y)  # Ball position

    return None  # Ball not found


In [13]:
# import time
# import gym
# import random
# import torch
# import numpy as np
# from collections import deque
# import matplotlib.pyplot as plt

# # Initialize Pong environment with human mode for visualization
# env = gym.make('Pong-v0', render_mode='rgb_array')
# env.seed(0)

# # Use GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print("Device: ", device)

# # Reset environment
# env.reset()

# # Function to describe game state
# def describe_game_state(player_paddle_y, opponent_paddle_y):
#     description = (f"Player paddle is at y = {player_paddle_y}, "
#                    f"opponent paddle is at y = {opponent_paddle_y}.")
#     return description

# # Dummy LLM function for generating action (replace with actual LLM function)
# def predict_action_llm(description):
#     # Example: Random action prediction based on the paddle positions
#     if "Player paddle" in description:
#         player_y = int(description.split("Player paddle is at y = ")[1].split(",")[0])
#         if player_y < 50:
#             return "down"
#         else:
#             return "up"
#     return "stay"

# # Map LLM's text-based action to game action
# def map_action(action_text):
#     if "up" in action_text.lower():
#         return 2  # Move paddle up
#     elif "down" in action_text.lower():
#         return 3  # Move paddle down
#     else:
#         return 0  # Stay still
# def predict_target_paddle_y(ball_x, ball_y, player_paddle_y):
#     # Prepare the game state description
#     description = f"Ball is at ({ball_x}, {ball_y}), player paddle is at y = {player_paddle_y}."
    
#     # Use the LLM to predict the target y-coordinate
#     # Replace this with your actual LLM model call
#     target_paddle_y = llm_predict_paddle_position(description)
    
#     return target_paddle_y

# def llm_predict_paddle_position(description):
#     # This is a placeholder for the LLM prediction
#     # In practice, you would use your LLM model to process the description
#     # and output a target y-coordinate for the paddle
    
#     # For demonstration, we'll align the paddle with the ball's y-position
#     import re
#     match = re.search(r'Ball is at \((\d+), (\d+)\)', description)
#     if match:
#         ball_y = int(match.group(2))
#         return ball_y
#     else:
#         # Default to current paddle position if parsing fails
#         return player_paddle_y

# # Update LLM to consider ball position when deciding the next action
# def predict_action_llm(ball_y, player_paddle_y):
#     if ball_y > player_paddle_y + 10:  # If the ball is below the paddle, move down
#         return "down"
#     elif ball_y < player_paddle_y - 10:  # If the ball is above the paddle, move up
#         return "up"
#     return "stay"


Device:  cpu


In [11]:
import time
import gym
import cv2
import numpy as np

# Initialize the environment properly with 'rgb_array' mode
env = gym.make('Pong-v4', render_mode='rgb_array')  # Switch to 'Pong-v4'
env.reset()

# Previous ball position for velocity calculation
previous_ball_x, previous_ball_y = None, None

def detect_ball(frame):
    if frame is not None:
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        _, thresh_frame = cv2.threshold(gray_frame, 150, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresh_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            if w < 15 and h < 15:
                return (x, y)
    return None

# Predict future ball position based on velocity
def predict_ball_position(ball_x, ball_y):
    global previous_ball_x, previous_ball_y

    if previous_ball_x is not None and previous_ball_y is not None:
        # Calculate ball velocity
        ball_velocity_x = ball_x - previous_ball_x
        ball_velocity_y = ball_y - previous_ball_y

        # Predict future position by adding velocity to current position
        future_ball_y = ball_y + ball_velocity_y * 5  # Look 5 frames ahead
    else:
        future_ball_y = ball_y  # No prediction available for first frame

    # Store the current position for the next frame
    previous_ball_x, previous_ball_y = ball_x, ball_y

    return future_ball_y

# Function to predict target paddle y-position
def predict_target_paddle_y(ball_x, ball_y, player_paddle_y):
    # Predict the future ball position
    future_ball_y = predict_ball_position(ball_x, ball_y)

    # Move the paddle towards the predicted future ball position
    return future_ball_y

# Main game loop with improved paddle movement
def random_play():
    step = 0
    score = 0
    player_paddle_y = 80  # Initial Y position

    env.reset()

    while True:
        step += 1

        # Render the game
        # time.sleep(0.05)  # Reduced delay for faster response
        frame = env.render()

        if frame is not None:
            print(f"Frame captured successfully: {frame.shape}")
        else:
            print("Failed to capture frame.")
            continue

        # Detect the ball's position
        ball_position = detect_ball(frame)
        if ball_position:
            ball_x, ball_y = ball_position
            print(f"Ball position: {ball_position}")

            # Predict the target paddle y-position using LLM
            target_paddle_y = predict_target_paddle_y(ball_x, ball_y, player_paddle_y)
            print(f"Target paddle y position predicted by LLM: {target_paddle_y}")

            # Move the paddle towards the target position
            if abs(player_paddle_y - target_paddle_y) < 5:
                action = 0  # Stay still if close enough to target
            elif player_paddle_y < target_paddle_y:
                action = 3  # Move down
            else:
                action = 2  # Move up
        else:
            print("Ball not found, staying still.")
            action = 0  # Stay still if ball not found

        # Perform the action in the environment
        state, reward, done, t, _ = env.step(action)

        # Update player paddle Y position based on action
        if action == 2:  # Moving up
            player_paddle_y = max(0, player_paddle_y - 20)  # Faster movement
        elif action == 3:  # Moving down
            player_paddle_y = min(210, player_paddle_y + 20)

        # If the game is done, end the loop
        if done:
            env.close()
            print("Your Score at the end of the game is: ", score)
            break

# Run the updated game loop
random_play()


Frame captured successfully: (210, 160, 3)
Ball not found, staying still.
Frame captured successfully: (210, 160, 3)
Ball not found, staying still.
Frame captured successfully: (210, 160, 3)
Ball not found, staying still.
Frame captured successfully: (210, 160, 3)
Ball not found, staying still.
Frame captured successfully: (210, 160, 3)
Ball not found, staying still.
Frame captured successfully: (210, 160, 3)
Ball not found, staying still.
Frame captured successfully: (210, 160, 3)
Ball not found, staying still.
Frame captured successfully: (210, 160, 3)
Ball not found, staying still.
Frame captured successfully: (210, 160, 3)
Ball not found, staying still.
Frame captured successfully: (210, 160, 3)
Ball not found, staying still.
Frame captured successfully: (210, 160, 3)
Ball not found, staying still.
Frame captured successfully: (210, 160, 3)
Ball not found, staying still.
Frame captured successfully: (210, 160, 3)
Ball not found, staying still.
Frame captured successfully: (210, 160

In [31]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load the DistilGPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained('distilgpt2')
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')

# Make sure to set the model to evaluation mode
model.eval()

# Function to make the LLM predict the target paddle y-coordinate using DistilGPT-2
def llm_predict_target_paddle_y(ball_x, ball_y, player_paddle_y):
    # Construct the game state description
    input_text = (f"The ball is at position ({ball_x}, {ball_y}). "
                  f"The paddle is at y = {player_paddle_y}. "
                  f"Predict the target y-coordinate where the paddle should move to intercept the ball.")
    
    # Tokenize the input text
    inputs = tokenizer(input_text, return_tensors="pt")
    
    # Generate the model's output, but now specify `max_new_tokens` to limit output length
    with torch.no_grad():
        outputs = model.generate(inputs['input_ids'], max_new_tokens=10)  # Set max_new_tokens to control output length
    
    # Decode the output and extract the generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"Model output: {generated_text}")

    # Extract the predicted y-coordinate from the generated text
    try:
        predicted_y = int(''.join(filter(str.isdigit, generated_text)))  # Extract numeric part from text
    except ValueError:
        predicted_y = player_paddle_y  # Fallback to current paddle position if parsing fails
    
    return predicted_y


In [ ]:
import time
import gym
import cv2
import numpy as np

# Initialize the Pong environment
env = gym.make('Pong-v4', render_mode='rgb_array')
env.reset()

def detect_ball(frame):
    if frame is not None:
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        _, thresh_frame = cv2.threshold(gray_frame, 150, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresh_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            if w < 15 and h < 15:
                return (x, y)
    return None

# Main game loop with DistilGPT-2 based paddle movement
def random_play():
    step = 0
    score = 0
    player_paddle_y = 80  # Initial paddle y position

    env.reset()

    while True:
        step += 1

        # Render the game
        time.sleep(0.05)  # Adjust the delay for smoother gameplay
        frame = env.render()

        if frame is not None:
            print(f"Frame captured successfully: {frame.shape}")
        else:
            print("Failed to capture frame.")
            continue

        # Detect the ball's position
        ball_position = detect_ball(frame)
        if ball_position:
            ball_x, ball_y = ball_position
            print(f"Ball position: {ball_position}")

            # Use the LLM to predict the target paddle y-position
            target_paddle_y = llm_predict_target_paddle_y(ball_x, ball_y, player_paddle_y)
            print(f"Target paddle y position predicted by LLM: {target_paddle_y}")

            # Move the paddle towards the target position
            if abs(player_paddle_y - target_paddle_y) < 5:
                action = 0  # Stay still if close enough to target
            elif player_paddle_y < target_paddle_y:
                action = 3  # Move down
            else:
                action = 2  # Move up
        else:
            print("Ball not found, staying still.")
            action = 0  # Stay still if ball not found

        # Perform the action in the environment
        state, reward, done, t, _ = env.step(action)

        # Update player paddle Y position based on action
        if action == 2:  # Moving up
            player_paddle_y = max(0, player_paddle_y - 20)  # Faster movement
        elif action == 3:  # Moving down
            player_paddle_y = min(210, player_paddle_y + 20)

        # If the game is done, end the loop
        if done:
            env.close()
            print("Your Score at the end of the game is: ", score)
            break

# Run the updated game loop
random_play()


Frame captured successfully: (210, 160, 3)
Ball not found, staying still.
Frame captured successfully: (210, 160, 3)
Ball not found, staying still.
Frame captured successfully: (210, 160, 3)
Ball not found, staying still.
Frame captured successfully: (210, 160, 3)
Ball not found, staying still.
Frame captured successfully: (210, 160, 3)
Ball not found, staying still.
Frame captured successfully: (210, 160, 3)
Ball not found, staying still.
Frame captured successfully: (210, 160, 3)
Ball not found, staying still.
Frame captured successfully: (210, 160, 3)
Ball not found, staying still.
Frame captured successfully: (210, 160, 3)
Ball not found, staying still.
Frame captured successfully: (210, 160, 3)
Ball not found, staying still.
Frame captured successfully: (210, 160, 3)
Ball not found, staying still.
Frame captured successfully: (210, 160, 3)
Ball not found, staying still.
Frame captured successfully: (210, 160, 3)
Ball not found, staying still.
Frame captured successfully: (210, 160

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Frame captured successfully: (210, 160, 3)
Ball not found, staying still.
Frame captured successfully: (210, 160, 3)
Ball not found, staying still.
Frame captured successfully: (210, 160, 3)
Ball not found, staying still.
Frame captured successfully: (210, 160, 3)
Ball position: (77, 116)
